# Business problem
### Background:
As the COVID-19 pandemic attack the U.S. economy, it provides the best time to put house on the market due to low inventory and high demand. The low interest rate has attract investors heating up the housing market. However, it is difficult for unfamiliar investor to collect housing information.

### Business Problem:
In this senerio, a newly-formed real estate company, ABC inc., is searching for investors who are inexperienced with the real estate market in Bronx. The company's goal is to collect information to attract new investors and assist new investor to make a profitable purchase.

The Conmpany's plan is to use clustering method to classfy neighboods in Bronx and recommending suitable price and venues for their new investors. 

### Data Selection
The data of housing price in Bronx can be access through the open source from New York City Government (link: https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page). It included all the rolling sales for past 12 months and it excluded the utility properties.

In order to analyze and designate location between different vecnues, we will evaluate amenities and essential facilities in each region using the data provided by FourSquare API interface. Then we will use the venue data  to generate a report with visualization. The venue data  from the API includes a neighborhood's name, neighborhood's latitude, lneighborhood's ongtitude, venue, venue's name, venue's latitude, and venue's logititude.
### Methodology:
In this case, I'll use data cleaning methods from IBM data science class and KNN and KMeansclustering for modeling.
The specific process is showned below

In [3]:
#Import libraries
import pandas as pd
import numpy as np
import pickle
import requests
import urllib
import json
import csv

import matplotlib.pyplot as plt 
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from folium import plugins

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup


print('succesfully import all the libraries...')

succesfully import all the libraries...


In [4]:
with open('nyu-2451-34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'nyu-2451-34572-geojson.json'